# TR Data Challenge 2023 - Data Analysis

This notebook reads and analyzes the TRDataChallenge2023.txt file containing legal documents.

In [ ]:
import json
import os
import pickle
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

## 1. Load the Data

In [ ]:
def load_tr_data(file_path='/mnt/d/TR-Project/TRDataChallenge2023.txt'):
    """
    Load TR data from the text file.
    """
    documents = []
    
    print(f"Loading data from: {file_path}")
    file_size_mb = os.path.getsize(file_path) / (1024*1024)
    print(f"File size: {file_size_mb:.2f} MB")
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if line.strip():
                try:
                    doc = json.loads(line.strip())
                    documents.append(doc)
                    
                    if (i + 1) % 5000 == 0:
                        print(f"Loaded {i + 1} lines...")
                        
                except json.JSONDecodeError:
                    continue
    
    print(f"Total documents loaded: {len(documents)}")
    return documents

# Load the data
documents = load_tr_data()

## 2. Explore Raw Data Structure

#### explore data structure in terminal before reading it</br>
```bash
head -n 1 TRDataChallenge2023.txt | python3 -m json.tool
```

In [ ]:
# Examine the structure of the first document
if documents:
    print("Keys in first document:")
    print(list(documents[0].keys()))
    
    print("\nFirst document:")
    print(json.dumps(documents[0], indent=2)[:1000] + "...")  # Show first 1000 chars

## 3. Convert to DataFrame

In [ ]:
def create_dataframe(documents):
    """
    Convert documents to a pandas DataFrame.
    """
    data = []
    
    for doc in documents:
        # Basic document info
        row = {
            'document_id': doc.get('documentId', ''),
            'postures': ', '.join(doc.get('postures', [])),
            'num_postures': len(doc.get('postures', [])),
            'num_sections': len(doc.get('sections', [])),
        }
        
        # Extract text content
        all_text = []
        section_headers = []
        
        for section in doc.get('sections', []):
            header = section.get('headtext', '')
            if header:
                section_headers.append(header)
            
            paragraphs = section.get('paragraphs', [])
            all_text.extend(paragraphs)
        
        row['section_headers'] = ' | '.join(section_headers)
        row['full_text'] = ' '.join(all_text)
        row['text_length'] = len(row['full_text'])
        row['word_count'] = len(row['full_text'].split())
        row['num_paragraphs'] = len(all_text)
        row['num_headers'] = len(section_headers)
        
        data.append(row)
    
    return pd.DataFrame(data)

# Create DataFrame
df = create_dataframe(documents)
print(f"DataFrame shape: {df.shape}")
df.head(2)

In [ ]:
from IPython.display import HTML, display

text = (
    "Examining the posture values in the dataframe reveals this is a classic "
    "multi-label classification problem. Each document can be assigned multiple "
    "posture labels simultaneously."
)

display(HTML(f'<span style="font-size: 150%;">{text}</span>'))

## 4. Data Analysis and Visualization

In [ ]:
# Basic statistics
print("Dataset Overview:")
print(df.info())
print("\nDescriptive Statistics:")
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
print()
print(df.describe().round(2))
print()

### 4.1 Multi-Label Postures Distribution Analysis

In [ ]:
plt.figure(figsize=(10, 6))

# Count number of documents per number of postures
posture_counts = df['num_postures'].value_counts().sort_index()
plt.bar(posture_counts.index, posture_counts.values)
plt.title('Distribution of Number of Postures per Document')
plt.xlabel('Number of Postures')
plt.ylabel('Number of Documents')
plt.show()

# Most common postures
all_postures = []
for postures in df['postures']:
    if postures:
        all_postures.extend([p.strip() for p in postures.split(',')])

posture_freq = pd.Series(all_postures).value_counts().head(10)
plt.figure(figsize=(12, 6))
posture_freq.plot(kind='bar')
plt.title('Top 10 Most Common Postures')
plt.xlabel('Posture')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
_counts = df['num_postures'].value_counts(dropna=False)
_pct = df['num_postures'].value_counts(dropna=False,normalize=True) 

pd.DataFrame({
    'count': _counts,
    'percentage': _pct
}).sort_index().style.format({'count':'{:,}','percentage':'{:.2%}'}).set_caption("Distribution of num_postures")\
    .set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '15px')]}])

<span style="font-weight: bold; font-size: 18px;">**Multi-Label Posture Distribution Analysis**<br>

Based on the distribution charts examining posture frequency and prevalence patterns, the following key insights emerge:

**Label Distribution Characteristics:**

<div style="margin-left: 20px;"> • <strong>Modal Distribution:</strong> The majority of documents exhibit 1-2 posture labels, indicating a relatively sparse multi-label structure</div></br>

<div style="margin-left: 20px;"> • <strong>Label Complexity:</strong> Documents can contain up to 4+ concurrent posture labels, representing the complexity of multi-label density</div></br>

<div style="margin-left: 20px;"> • <strong>Unlabeled Data:</strong> Approximately 1,000 documents lack posture annotations (num_postures=0), requiring exclusion from the training dataset to prevent noise injection</div></br>

<div style="margin-left: 20px;"> • <strong>Severe Label Imbalance:</strong> "On Appeal" and "Appellate Review" dominate the dataset as the most prevalent posture labels, while numerous categories exhibit extremely low frequency, creating a highly imbalanced multi-label distribution that poses significant challenges for model training</div></br>

<div style="margin-left: 20px;"> • <strong>Class Imbalance Mitigation Strategies:</strong> </br>

The substantial disparity between dominant and rare categories necessitates implementation of specialized techniques such as:</br>

  <div style="margin-left: 40px;">- Undersampling of overrepresented classes</div></br>

  <div style="margin-left: 40px;">- Class-weighted loss functions to amplify learning signals from minority categories</div></br>

  <div style="margin-left: 40px;">- Focal loss or other imbalance-aware optimization approaches</div></br>

</div>

<div style="margin-left: 20px;"> • <strong>Training Implications:</strong> Without proper handling of this imbalance, the model may develop bias toward frequent categories while failing to adequately learn rare but potentially important legal postures</div>

**Data Quality Implications:**
The identified unlabeled instances necessitate data preprocessing to ensure training dataset integrity and model performance optimization.

</span>

In [ ]:
# Prepare the labels - convert postures to a list format
def prepare_labels(postures_str):
    """Convert posture string to list of postures"""
    if pd.isna(postures_str) or postures_str == '':
        return []
    return [p.strip() for p in postures_str.split(',') if p.strip()]

df['posture_list'] = df['postures'].apply(prepare_labels)

# Remove documents with no postures
df_temp = df[df['posture_list'].apply(len) > 0].copy()
print("{:<30}{:<20,}".format("Documents with postures: ", len(df_temp)))
print("{:<30}{:<20}".format("Documents without postures: ", len(df)-len(df_temp)))

In [ ]:
# Analyze posture distribution
all_postures_ml = []
for postures in df_temp['posture_list']:
    all_postures_ml.extend(postures)

posture_counts = pd.Series(all_postures_ml).value_counts()
print(f"\nTotal unique postures: {len(posture_counts)}")
print()
print(f"Most common postures:")
print(posture_counts.head(15))

In [ ]:
# Filter to most common postures (those appearing in at least 100 documents)
min_frequency = 100
common_postures = posture_counts[posture_counts >= min_frequency].index.tolist()
print(f"\nPostures with >= {min_frequency} occurrences: {len(common_postures)}")
print(common_postures)

In [ ]:
# Filter documents to only include those with common postures
def filter_common_postures(posture_list, common_postures):
    """Keep only postures that are in the common_postures list"""
    return [p for p in posture_list if p in common_postures]

df_temp['filtered_postures'] = df_temp['posture_list'].apply(
    lambda x: filter_common_postures(x, common_postures)
)

# Remove documents that have no common postures after filtering
df_ml = df_temp[df_temp['filtered_postures'].apply(len) > 0].copy()
print(f"Documents after filtering to common postures: {len(df_ml)}")

In [ ]:
unique_postures_raw = len(set(itertools.chain.from_iterable(df['posture_list'])))
unique_postures_temp = len(set(itertools.chain.from_iterable(df_temp['posture_list'])))
unique_postures_ml = len(set(itertools.chain.from_iterable(df_ml['filtered_postures'])))

pd.DataFrame({
    "--": ["raw_document", "Removing Missing Postures", "Filter to Common Postures"],
    "sample size": [df.shape[0], df_temp.shape[0], df_ml.shape[0]],
    "Total unique postures": [unique_postures_raw, unique_postures_temp, unique_postures_ml]
}).style.format({'sample size':'{:,}'})

### 4.2 Text Length Distribution Analysis

<span style="color: red; font-weight: bold; font-size: 18px;"> **Text Length Analysis for Model Selection and Optimization**</br>

Analyzing text length distribution is a critical preprocessing step that directly influences model architecture selection and performance optimization. When deploying Large Language Models (LLMs) for this multi-label classification task, context length becomes a pivotal consideration due to several key factors:

**Context Length Constraints:** </br>Each LLM architecture has a predefined maximum context length (e.g., 512 tokens for BERT, 4096 for Longformer, 8192 for MordernBERT, 10000+ for many decoder models(Llama, Mistral, Qwen family etc)). Understanding our data's text length distribution enables informed model selection that balances computational efficiency with information preservation.

**Resource Optimization:**</br> Oversized context windows lead to unnecessary computational overhead and increased memory consumption without performance gains. Conversely, insufficient context length results in critical information truncation, potentially degrading model accuracy.

**Strategic Implications:** </br>By examining the statistical distribution of text lengths across our dataset, we can:

Select appropriate model architectures with optimal context lengths
Implement efficient text preprocessing strategies (chunking, summarization, or truncation)
Estimate computational requirements and training costs
Ensure minimal information loss during model input preparation
This analysis forms the foundation for making data-driven decisions regarding model selection and hyperparameter configuration. </span>

In [ ]:
# Text length distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(df['text_length'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution of Text Length')
plt.xlabel('Text Length (characters)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(df['word_count'], bins=50, alpha=0.7, edgecolor='black', color='orange')
plt.title('Distribution of Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

<span style="font-weight: bold; font-size: 18px;"> **Accurate Text Length Measurement Using Subword Tokenization**</br>

The initial text length analysis employed a basic word-counting approach using whitespace as delimiters. However, this naive methodology fails to accurately represent how Large Language Models process and interpret textual input, leading to potential misestimation of context requirements.

**Tokenization Discrepancy:**</br> LLMs do not process text at the word level as humans do. Instead, they utilize sophisticated subword tokenization algorithms including:

<div style="margin-left: 20px;">1. Byte Pair Encoding (BPE): Used by GPT models</div>
<div style="margin-left: 20px;">2. WordPiece: Employed by BERT variants</div>
<div style="margin-left: 20px;">3. SentencePiece: Utilized by T5 and many multilingual models</div>

**Technical Implementation:**</br> To obtain precise context length measurements that align with actual LLM processing, we implement tokenization using ModernBERT, which employs BPE (Byte Pair Encoding) for subword segmentation. This approach provides several advantages:

<div style="margin-left: 20px;">1. Accurate Token Counting: Measures text length in the same units that the model processes</div>
<div style="margin-left: 20px;">2. Context Window Optimization: Enables precise estimation of how much text fits within model constraints</div>
<div style="margin-left: 20px;">3. Model-Agnostic Approach: The methodology can be adapted for different tokenizers (e.g., GPT-2/3/4 or many other LLMs' tokenizer) based on target model selection</div>


</span>

### text length distribution using subword tokenization

In [ ]:
from transformers import AutoTokenizer
model_path = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
from datasets import Dataset
# text_df=df["full_text"].to_frame("text")
text_df=df_ml.loc[:,["full_text","postures"]]
hf=Dataset.from_pandas(text_df)

def compute_lenth(example):
    return {"text_length":len(example["input_ids"])}
hf=hf.map(lambda x: tokenizer(x["full_text"]),batched=True)
hf=hf.map(compute_lenth)
df0=hf.to_pandas()

In [ ]:
# Text length distribution
plt.figure(figsize=(12, 6))

plt.hist(df0['text_length'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution of Text Length')
plt.xlabel('Text Length : Subword Tokenization(BPE)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

<span style="font-weight: bold; font-size: 18px;">**Subword Tokenization Analysis Results and Model Selection**</br>

The subword tokenization analysis reveals that the average text length across the corpus is 3,826 tokens. When considering ModernBERT as our target language model, which supports a maximum context length of 8,192 tokens, this configuration provides excellent coverage for our dataset.

Key Findings:

<div style="margin-left: 20px;"> •  Average token length: 3,826 tokens per document</div>
<div style="margin-left: 20px;"> •  Model capacity: ModernBERT's 8,192-token context window</div>
<div style="margin-left: 20px;"> •  Coverage efficiency: Approximately 90% of documents (up to 8,143 tokens) can be processed without truncation</div>


</span>

## 5. split the data into training, validation and test set

In [ ]:
## Multi-label Classification Setup

# Create binary label matrix using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_multilabel = mlb.fit_transform(df_ml['filtered_postures'])

print(f"Label matrix shape: {y_multilabel.shape}")
print(f"Labels: {mlb.classes_}")

In [ ]:
# Prepare text data
X_text = df_ml['full_text'].values

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(
    X_text, y_multilabel, 
    test_size=0.3, # 30% for temp (which will be split into val and test)
    random_state=42, 
    stratify=None
)

 # Split temp into validation and test (50-50 split of the 30%)
# # This gives us 15% each
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42, 
    stratify=None
)

print("{:<18}{:<10,}{:<15.0%}".format("Total samples:",len(df_ml),1))
print("{:<18}{:<10,}{:<15.0%}".format("Training set:",len(X_train),len(X_train)/len(df_ml)))
print("{:<18}{:<10,}{:<15.0%}".format("Validation set:",len(X_val),len(X_val)/len(df_ml)))
print("{:<18}{:<10,}{:<15.0%}".format("Test set:",len(X_test),len(X_test)/len(df_ml)))

In [ ]:
# Check label distribution
train_label_sums = y_train.sum(axis=0)
val_label_sums = y_val.sum(axis=0)
test_label_sums = y_test.sum(axis=0)

print("\nLabel distribution in training set:")
print()
for i, label in enumerate(mlb.classes_):
    print(f"{label}: {train_label_sums[i]} ({train_label_sums[i]/len(y_train)*100:.1f}%)")

In [ ]:
# Check label distribution
def label_dist(y_train, data_split="training"):
    train_label_sums = y_train.sum(axis=0)

    # Get top 5 most frequent labels and their indices
    top5_indices = train_label_sums.argsort()[-5:][::-1]
    top5_labels = [mlb.classes_[i] for i in top5_indices]
    top5_counts = train_label_sums[top5_indices]

    print(f"\nTop 5 most frequent labels in {data_split} set:")
    print()
    for idx, label in zip(top5_indices, top5_labels):
        count = train_label_sums[idx]
        percent = count / len(y_train) 
        # print(f"{label}: {count} ({percent:.1f}%)")
        print("{:<50}{:<10,}{:<15.1%}".format(f"{label}:", count, percent))

In [ ]:
label_dist(y_train, data_split="training")

In [ ]:
label_dist(y_val, data_split="validation")

In [ ]:
label_dist(y_test, data_split="test")

## 6. Save Preprocess Data

In [ ]:
## save preprocess data
saved_data=os.path.join(os.getcwd(), 'processed_data')
os.makedirs(saved_data, exist_ok=True)
# Save using pickle
with open(os.path.join(saved_data,'train_arrays.pkl'), 'wb') as f:
    pickle.dump({'X_train': X_train, 'y_train': y_train}, f)

with open(os.path.join(saved_data,'val_arrays.pkl'), 'wb') as f:
    pickle.dump({'X_val': X_val, 'y_val': y_val}, f)

with open(os.path.join(saved_data,'test_arrays.pkl'), 'wb') as f:
    pickle.dump({'X_test': X_test, 'y_test': y_test}, f)

with open(os.path.join(saved_data,'class_name.pkl'), 'wb') as f:
    pickle.dump({'class_name': mlb.classes_}, f)

print("All arrays saved with pickle!")

# To load later:
# with open(os.path.join(saved_data,'train_arrays.pkl'), 'rb') as f:
#     train_data = pickle.load(f)
#     X_train = train_data['X_train']
#     y_train = train_data['y_train']
#     label_train = train_data['label_train']